**Setting system path variable**

In [0]:
import os
import sys
print(os.getcwd())
project_pth=os.path.join(os.getcwd(),"..","..")
sys.path.append(project_pth)
from utils.reusable import reusable

/Workspace/Users/priyabaskar2109@gmail.com/etl_dab/src/silver


In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

##**DimUser**##

In [0]:
df_user=spark.readStream.format("cloudFiles")\
                        .option("cloudFiles.format","parquet")\
                        .option("cloudFiles.schemaLocation","abfss://silver@varunstorageaccount.dfs.core.windows.net/DimUser/schema")\
                        .option("cloudFiles.schemaEvolutionMode","rescue")\
                        .load("abfss://bronze@varunstorageaccount.dfs.core.windows.net/DimUser")

In [0]:
display(df_user,checkpointLocation="abfss://silver@varunstorageaccount.dfs.core.windows.net/DimUser/checkpoint")

In [0]:
df_user=df_user.withColumn("user_name",upper(col("user_name")))
display(df_user,checkpointLocation="abfss://silver@varunstorageaccount.dfs.core.windows.net/DimUser/displaycheckpoint")

In [0]:
df_user_obj=reusable()
df_user=df_user_obj.dropColumns(df_user,["_rescued_data"])
df_user=df_user.dropDuplicates(["user_id"])
display(df_user,checkpointLocation="abfss://silver@varunstorageaccount.dfs.core.windows.net/DimUser/checkpointdropcol")

In [0]:
df_user.writeStream.format("delta")\
                   .outputMode("append")\
                   .option("checkpointLocation","abfss://silver@varunstorageaccount.dfs.core.windows.net/DimUser/checkpointWrite")\
                   .trigger(once=True)\
                   .option("path","abfss://silver@varunstorageaccount.dfs.core.windows.net/DimUser/data")\
                   .toTable("etl_cata.silver.DimUser")

##**DimArtists**##

In [0]:
df_artist=spark.readStream.format("cloudFiles")\
                          .option("cloudFiles.format","parquet")\
                          .option("cloudFiles.schemaLocation","abfss://silver@varunstorageaccount.dfs.core.windows.net/DimArtist/Schema")\
                          .option("cloudFiles.schemaEvolutionMode","addNewColumns")\
                          .load("abfss://bronze@varunstorageaccount.dfs.core.windows.net/DimArtist")

In [0]:
display(df_artist, checkpointLocation="abfss://silver@varunstorageaccount.dfs.core.windows.net/DimArtist/checkpoint")

In [0]:
df_obj=reusable()

In [0]:
df_artist=df_obj.dropColumns(df_artist,["_rescued_data"])

In [0]:
df_artist.writeStream.format("delta")\
                     .outputMode("append")\
                     .option("checkpointLocation","abfss://silver@varunstorageaccount.dfs.core.windows.net/DimArtist/checkpointWrite")\
                     .trigger(once=True)\
                     .option("path","abfss://silver@varunstorageaccount.dfs.core.windows.net/DimArtist/data")\
                     .toTable("etl_cata.silver.DimArtist")

## **DimTrack**##

In [0]:
df_track=spark.readStream.format("cloudFiles")\
                         .option("cloudFiles.format","parquet")\
                         .option("cloudFiles.schemaLocation","abfss://silver@varunstorageaccount.dfs.core.windows.net/DimTrack/Schema")\
                         .option("cloudFiles.schemaEvolutionMode","addNewColumns")\
                         .load("abfss://bronze@varunstorageaccount.dfs.core.windows.net/DimTrack")
display(df_track, checkpointLocation="abfss://silver@varunstorageaccount.dfs.core.windows.net/DimTrack/checkpoint")

In [0]:
df_track=df_track.withColumn("duration_len",when(col("duration_sec")<150,"short").when((col("duration_sec")>150)&(col("duration_sec")<300),"medium").otherwise("long"))
df_track=df_track.withColumn("track_name",regexp_replace(col("track_name"),'-',' '))
df_track=reusable().dropColumns(df_track,["_rescued_data"])
display(df_track,checkpointLocation="abfss://silver@varunstorageaccount.dfs.core.windows.net/DimTrack/checkpointdisplay")

In [0]:
df_track.writeStream.format("delta")\
                    .outputMode("append")\
                    .option("checkpointLocation","abfss://silver@varunstorageaccount.dfs.core.windows.net/DimTrack/checkpointWrite")\
                    .trigger(once=True)\
                    .option("path","abfss://silver@varunstorageaccount.dfs.core.windows.net/DimTrack/data")\
                    .toTable("etl_cata.silver.DimTrack")

##**DimDate**##

In [0]:
df_date=spark.readStream.format("cloudFiles")\
                        .option("cloudFiles.format","parquet")\
                        .option("cloudFiles.schemaLocation","abfss://silver@varunstorageaccount.dfs.core.windows.net/DimDate/Schema")\
                        .option("cloudFiles.schemaEvolutionMode","addNewColumns")\
                        .load("abfss://bronze@varunstorageaccount.dfs.core.windows.net/DimDate")
df_date=reusable().dropColumns(df_date,["_rescued_data"])
df_date.writeStream.format("delta")\
                .outputMode("append")\
                .option("checkpointLocation","abfss://silver@varunstorageaccount.dfs.core.windows.net/DimDate/checkpointWrite")\
                .trigger(once=True)\
                .option("path","abfss://silver@varunstorageaccount.dfs.core.windows.net/DimDate/data")\
                .toTable("etl_cata.silver.DimDate")

##**FactStream**##

In [0]:
df_fact=spark.readStream.format("cloudFiles")\
                        .option("cloudFiles.format","parquet")\
                        .option("cloudFiles.schemaLocation","abfss://silver@varunstorageaccount.dfs.core.windows.net/FactStream/Schema")\
                        .option("cloudFiles.schemaEvolutionMode","addNewColumns")\
                        .load("abfss://bronze@varunstorageaccount.dfs.core.windows.net/FactStream")
df_fact=reusable().dropColumns(df_fact,["_rescued_data"])
df_fact.writeStream.format("delta")\
                .outputMode("append")\
                .option("checkpointLocation","abfss://silver@varunstorageaccount.dfs.core.windows.net/FactStream/checkpointWrite")\
                .trigger(once=True)\
                .option("path","abfss://silver@varunstorageaccount.dfs.core.windows.net/FactStream/data")\
                .toTable("etl_cata.silver.FactStream")